## Import data and packages

In [15]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import numpy as np 
import json
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split


# import data
df_base = pd.read_csv('data/Base.csv') 

# import feature data types
with open('python_scripts/data_types.json') as f:
    data_types = json.load(f) 

# encoding source {'INTERNET', 'TELEAPP'} into source_is_internet_not_teleapp
df_base['source_is_internet_not_teleapp'] = (df_base['source'] == 'INTERNET').astype(int)
df_base.drop('source', axis=1, inplace=True)


# recording which variables have missing values recorded as -1
missing_values = ['prev_address_months_count',
                  'current_address_months_count',
                  'bank_months_count',
                  'session_length_in_minutes',
                  'device_distinct_emails_8w']

### Divide dataset into training set and testing set

In [16]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_base.drop('fraud_bool', axis=1), df_base.fraud_bool, test_size=0.2, random_state=0)


### Deal with missing values

### Encode and simplfy categorical features

### Scaling for numerical features